# Copy instead of modifying to preserve attack parameters

In [17]:
AGENT_NAME = '20 bin PPO 500 results\default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500.zip'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
#RUNS = 5
TRIAL = 1
SAVE_DIR = r'20 bin PPO 500 results\random noise eps vs ASR results' + '/'

In [2]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

import pandas as pd
import numpy as np


import KBMproject.utilities as utils

from joblib import Parallel, delayed

%matplotlib inline

In [3]:
schema = DataSet.get_schema(DATASET_NAME)

In [4]:
testilons = np.arange(0.02, 0.21, 0.01)
RUNS = len(testilons)

Define RL agent

In [5]:
agents = []
for _ in range (RUNS):
    agents.append(PPO.load(AGENT_NAME))

Create environments

In [6]:
envs = []
for _ in range (RUNS):
    envs.append(utils.make_discrete_env(schema=schema,  
                            action_bins=agents[0].action_space[0].n,
                            seed=42))

In [7]:
cols = utils.make_discrete_env(schema=schema,  
                            action_bins=agents[0].action_space[0].n,
                            seed=42).observation_names

In [8]:
observation_masks = np.ones(agents[0].observation_space.shape)
observation_masks[0:6] = 0 #mask time features
print('masked features:')
cols[0][0:6]

masked features:


['month_cos',
 'month_sin',
 'day_type_cos',
 'day_type_sin',
 'hour_cos',
 'hour_sin']

In [9]:
%%time
%%capture
results = Parallel(n_jobs=RUNS, verbose=10, prefer='threads')(delayed(
    utils.eval_rand_attack)(agent, env, eps) for agent, env, eps in zip(agents, envs, testilons)) 


CPU times: total: 23min 8s
Wall time: 13min 24s


Results is a list of tupples for each run, of the format(KPIs, observations, perturbed observations, epsilons)

In [10]:
kpis = [results[i][0] for i in range(len(results))]
df_kpis = pd.concat(kpis, axis='columns',keys=testilons)

In [11]:
#df_kpis[['mean', 'std', 'variance']] = df_kpis.agg(['mean','std', 'var'], axis='columns')

In [12]:
df_kpis

,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20
cost_function,,,,,,,,,,,,,,,,,,,
annual_peak_average,1.009338,1.100310,1.022187,1.009338,1.009338,1.009338,1.258521,1.258520,1.258523,1.102279,1.258526,1.063141,1.009338,1.044878,1.034155,1.063142,1.027269,1.258519,1.009338
carbon_emissions_total,0.869291,0.869456,0.869764,0.871023,0.872081,0.873338,0.871641,0.873836,0.873605,0.872991,0.874863,0.875812,0.875577,0.876364,0.878422,0.879065,0.878182,0.880392,0.880601
cost_total,0.786721,0.786682,0.786108,0.788184,0.788692,0.788831,0.786811,0.789103,0.789127,0.788254,0.789706,0.790473,0.790993,0.792339,0.791756,0.793283,0.793391,0.794458,0.794217
daily_one_minus_load_factor_average,1.088789,1.092079,1.094508,1.115129,1.086207,1.085199,1.079234,1.084362,1.067334,1.100333,1.122169,1.088168,1.103053,1.089385,1.064829,1.076129,1.192223,1.074436,1.105581
daily_peak_average,0.891380,0.892572,0.890590,0.898505,0.899425,0.901592,0.898813,0.903778,0.902117,0.908287,0.912164,0.907634,0.905140,0.919626,0.903852,0.915595,0.923988,0.927165,0.923519
electricity_consumption_total,0.879570,0.879497,0.880311,0.881323,0.882245,0.883534,0.882493,0.883975,0.884529,0.884212,0.886087,0.887224,0.886768,0.887289,0.889908,0.890430,0.890084,0.892706,0.892528
monthly_one_minus_load_factor_average,0.981439,0.986953,0.985185,0.977918,0.982898,0.971843,0.986605,0.986324,0.990302,0.990457,0.994079,0.974196,0.979635,0.984337,0.983827,0.987470,0.979061,0.994840,0.983533
ramping_average,1.099021,1.099375,1.101486,1.106716,1.106670,1.112332,1.110655,1.112969,1.118226,1.118907,1.121835,1.126019,1.127060,1.131485,1.128311,1.150000,1.145033,1.157108,1.144700
zero_net_energy,1.107648,1.107684,1.107821,1.107800,1.107947,1.108162,1.108216,1.108169,1.108417,1.108269,1.108140,1.108020,1.108154,1.107868,1.107489,1.107702,1.107686,1.108318,1.108701


In [18]:
df_kpis.to_csv(SAVE_DIR + f'{RUNS} run KPIs {TRIAL}.csv')

FGSM using the same parameters in a whitebox attack has an ASR of 0.7.

Increasing the number of restarts decreases the ASR, likely because of differences between the surrogate and vicitm models

In [19]:
ASRs = [results[i][3] for i in range(len(results))]
#print(f'For 5 runs of the random attack \nthe mean ASR is: {np.mean(ASRs):.3f}\nthe STD is: {np.std(ASRs):.3f}')

In [20]:
ASRs

[0.053767123287671234,
 0.08184931506849315,
 0.10308219178082192,
 0.12488584474885844,
 0.1517123287671233,
 0.17294520547945205,
 0.19429223744292237,
 0.21244292237442922,
 0.2408675799086758,
 0.2536529680365297,
 0.2737442922374429,
 0.2863013698630137,
 0.30468036529680365,
 0.32054794520547947,
 0.33550228310502284,
 0.3487442922374429,
 0.35924657534246573,
 0.3817351598173516,
 0.3863013698630137]

In [21]:
pd.DataFrame({'ASRs':ASRs}, index=testilons,).to_csv(SAVE_DIR + f'ASR for varied epsilons {TRIAL}.csv')

In [22]:
for run in range(RUNS):
        pd.DataFrame(results[run][1],columns=cols,).to_csv(SAVE_DIR+f'run {run} obs {TRIAL}.csv',)
        pd.DataFrame(results[run][2],columns=cols,).to_csv(SAVE_DIR+f'run {run} adv obs {TRIAL}.csv')

ValueError: Shape of passed values is (8759, 1), indices imply (8759, 31)